In [1]:
import ray
import time
import numpy as np
%alias_magic t timeit
ray.init(num_cpus = 24)


/home/adkern/miniconda3/envs/ray/lib/python3.9/site-packages/redis/connection.py:72: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)


Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.


{'node_ip_address': '184.171.122.115',
 'raylet_ip_address': '184.171.122.115',
 'redis_address': '184.171.122.115:6379',
 'object_store_address': '/tmp/ray/session_2021-11-17_22-09-35_273562_64443/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-11-17_22-09-35_273562_64443/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2021-11-17_22-09-35_273562_64443',
 'metrics_export_port': 57988,
 'node_id': '941d03d4cf4e8c3d58ba5e9920cb1777abaa71cf84c8af248a8706af'}

### numpy baseline

In [3]:
# random large matrix
x = np.random.random((1_000_000,1000))


In [4]:
%%t
res2 = [ np.sum(x[:, i]) for i in range(x.shape[1])]

8.83 s ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### using ray

In [5]:
# set up so-called remote function to broadcast work
@ray.remote
def remote_sum(array_id, idx):
    return np.sum(array_id[:,idx])

# broadcast random matrix to share among workers
obj_ref1 = ray.put(x)


In [6]:
%%t

result_ids = [ remote_sum.remote(obj_ref1, i) for i in range(x.shape[1]) ]
results = ray.get(result_ids)


(remote_sum pid=64537) 
1.22 s ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
